In [6]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.DataIO import DataIO
from skopt.space import Real, Integer, Categorical
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender_Multiple_ICM_stacked
np.random.seed(2498)
from Recommenders.FactorizationMachines.LightFMRecommender import LightFMCFRecommender

columns_name = ['user_id','item_id','impression_list','data']

dataset_l = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_length.csv', sep=',')
dataset_t = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_type.csv', sep=',')
dataset = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/interactions_and_impressions.csv', sep=',')
test_users = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_target_users_test.csv',sep = ',')
dataset.columns = columns_name

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
from Data_manager.Build_Matrix.Data_import import build_ICM_type,build_ICM_lengh_kmeans_3,build_URM_ICM_onlyURM_item,build_URM_impression
import scipy.sparse as sps
URM_ALL,ICM=build_URM_ICM_onlyURM_item(dataset, dataset_t, dataset_l)
ICM_TYPE   =build_ICM_type(dataset_t)
ICM_LENGHT =build_ICM_lengh_kmeans_3(dataset_l,n_clusters = 5)
ICM_genre_subgenre = sps.hstack((ICM_TYPE, ICM_LENGHT));

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_ALL, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 697 ( 1.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 308 ( 0.7%) Users that have less than 1 test interactions


In [9]:
df = dataset.copy()
df_impressions = df.drop(columns = ['data','item_id'])
df_impressions.dropna(inplace=True)
    
df_impressions = df_impressions.groupby(by=['user_id'])['impression_list'].apply(list).reset_index()

def helper(list_string):
    data = []
    for string in list_string:
        stuff = string.split(',')
        for s in stuff:
            int(s)
            data.append(int(s))
    return data

df_impressions['impression_list']= df_impressions['impression_list'].apply(helper)
df_impressions = df_impressions.groupby(by=['user_id'])['impression_list'].apply(np.array).reset_index()
df_impressions['impression_list'] = df_impressions['impression_list'].apply(np.concatenate)
df_impressions['impression_list'] = df_impressions['impression_list'].apply(np.unique)
num_users = 41629
num_items = 24507


# URM_impression = sps.csr_matrix(A)


In [10]:
A=[]
for i in df_impressions['user_id']:
    for j in df_impressions.iloc[i]['impression_list']:
        A.append({'user_id' : i , 'item_id' : j , 'data' : 1})
for i in range(0,41629):
    A.append({'user_id' : i , 'item_id' : 0 , 'data' : 0})

In [11]:
pippo = pd.DataFrame(A)
unique_items = pippo.item_id.unique()
num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()
mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

In [12]:
pippo = pd.merge(left=pippo, 
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

In [13]:
num_users = 41629
num_items = max_item_id

In [14]:
pippo

,user_id,item_id,data,mapped_item_id
0,0,0,1,0
1,191,0,1,0
2,1343,0,1,0
3,1849,0,1,0
4,1875,0,1,0
...,...,...,...,...
1646666,35938,21596,1,21639
1646667,35955,21610,1,21640
1646668,35977,21622,1,21641
1646669,35988,21627,1,21642


In [15]:
num_items=len(pippo.item_id.unique())

In [16]:
urm_impression = sps.csr_matrix((pippo.data,(pippo.user_id,pippo.mapped_item_id)),shape=(num_users, num_items))

In [17]:
from Recommenders.FactorizationMachines.LightFMRecommender import LightFMUserHybridRecommender
rec = LightFMUserHybridRecommender(URM_train,urm_impression)

LightFMUserHybridRecommender: UCM Detected 5610 (13.5%) cold users.


In [18]:
import os

output_folder_path = "result_LightFMCFRecommender"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 500
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

hyperparameters_range_dictionary = {
                "epochs": Categorical([30]),
                "n_components": Integer(1, 200),
                "loss": Categorical(['bpr', 'warp', 'warp-kos']),
                "sgd_mode": Categorical(['adagrad', 'adadelta']),
                "learning_rate": Real(low = 1e-6, high = 1e-1, prior = 'log-uniform'),
                "item_alpha": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
                "user_alpha": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
            }

# earlystopping_keywargs = {"validation_every_n": 25,
#                               "stop_on_validation": True,
#                               "evaluator_object": evaluator_test,
#                               "lower_validations_allowed": 1,
#                               "validation_metric": metric_to_optimize,
#                               }
recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS = [URM_train,urm_impression],
                CONSTRUCTOR_KEYWORD_ARGS = {},
                FIT_POSITIONAL_ARGS = [],
                FIT_KEYWORD_ARGS = {},
                EARLYSTOPPING_KEYWORD_ARGS = {},
            )

recommender_class = LightFMUserHybridRecommender


hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                        evaluator_validation=evaluator_test,
                                        evaluator_test=evaluator_test)

In [19]:
hyperparameterSearch.search(recommender_input_args,
                    hyperparameter_search_space = hyperparameters_range_dictionary,
                    n_cases = n_cases,
                    n_random_starts = n_random_starts,
                    save_model = "last",
                    output_folder_path = output_folder_path, # Where to save the results
                    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                    metric_to_optimize = metric_to_optimize,
                    cutoff_to_optimize = cutoff_to_optimize,
                    resume_from_saved = True,
                    recommender_input_args_last_test=recommender_input_args
                    )

SearchBayesianSkopt: Resuming 'LightFMUserHybridRecommender' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'epochs': 30, 'n_components': 102, 'loss': 'warp', 'sgd_mode': 'adadelta', 'learning_rate': 0.028419556427889625, 'item_alpha': 5.39366020797983e-05, 'user_alpha': 0.008943349638718973}
LightFMUserHybridRecommender: UCM Detected 5610 (13.5%) cold users.
LightFMUserHybridRecommender: Epoch 1 of 30. Elapsed time 8.68 min
LightFMUserHybridRecommender: Epoch 2 of 30. Elapsed time 16.98 min
LightFMUserHybridRecommender: Epoch 3 of 30. Elapsed time 24.94 min
LightFMUserHybridRecommender: Epoch 4 of 30. Elapsed time 32.52 min
LightFMUserHybridRecommender: Epoch 5 of 30. Elapsed time 40.01 min
LightFMUserHybridRecommender: Epoch 6 of 30. Elapsed time 47.37 min
LightFMUserHybridRecommender: Epoch 7 of 30. Elapsed time 54.60 min
LightFMUserHybridRecommender: Epoch 8 of 30. Elapsed time 1.03 hour
LightFMUserH